### VERİ SETİ HAKKINDA

Walmart, Amerika Birleşik Devletleri merkezli dünya çapında şubelere sahip bir perakende mağazalar zinciridir.
Çeşitli mağaza formatları ve online satış kanallarıyla geniş bir ürün yelpazesi sunar. Bu projede Walmart'ın haftalık satışlarını analiz edeceğiz. 

###### Verimizdeki değişkenleri kısaca açıklayacak olursak:

- Store: Mağaza numarası.
- Date: Tarih.
- Temperature: Ortalama haftalık sıcaklık (Fahrenheit).
- Fuel_Price: Yakıt fiyatı (galon başına).
- MarkDown1 - MarkDown5: Farklı türdeki indirimler.
- CPI: Tüketici Fiyat Endeksi / Enflasyon. 
- Unemployment: İşsizlik oranı.
- IsHoliday: Haftanın tatil dönemi olup olmadığını belirten bir bayrak (True/False).
- Type: Mağaza tipi (A, B, C).
- Size: Mağaza büyüklüğü (kare feet olarak).
- Dept: Departman numarası.
- Weekly_Sales: Haftalık satış miktarı.

Bu değişkenler feature.csv, train.csv, store.csv dosyalarında dağınık şekilde bulunuyor. Bu bölümde bu dosyaları inceleyip veriyi analiz için uygun hale getireceğiz.

#### KÜTÜPHANE TANIMLAMALARI

In [1]:
import pandas as pd
import numpy as np

#### TERMİNAL DÜZENLEMELERİ

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x) #virgülden sonra üç basamak göster

#### DATASET YÜKLENMESİ VE BİRLEŞTİRİLMESİ

In [3]:
# ayrı dosyalardaki verileri okuyalım.
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')
train = pd.read_csv('train.csv')

In [4]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.500,False
1,1,1,2010-02-12,46039.490,True
2,1,1,2010-02-19,41595.550,False
3,1,1,2010-02-26,19403.540,False
4,1,1,2010-03-05,21827.900,False


In [5]:
train.shape

(421570, 5)

In [6]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.310,2.572,NaN,NaN,NaN,NaN,NaN,211.096,8.106,False
1,1,2010-02-12,38.510,2.548,NaN,NaN,NaN,NaN,NaN,211.242,8.106,True
2,1,2010-02-19,39.930,2.514,NaN,NaN,NaN,NaN,NaN,211.289,8.106,False
3,1,2010-02-26,46.630,2.561,NaN,NaN,NaN,NaN,NaN,211.320,8.106,False
4,1,2010-03-05,46.500,2.625,NaN,NaN,NaN,NaN,NaN,211.350,8.106,False


In [7]:
features.shape

(8190, 12)

In [8]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [9]:
stores.shape

(45, 3)

In [10]:
# incelediğimizde IsHoliday değişkeni hem features hem train setinde var.
# Birleştirirken belirleyici olarak kullanmayacağım için birini siliyorum.
features.drop(columns= "IsHoliday", inplace=True)
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,42.310,2.572,NaN,NaN,NaN,NaN,NaN,211.096,8.106
1,1,2010-02-12,38.510,2.548,NaN,NaN,NaN,NaN,NaN,211.242,8.106
2,1,2010-02-19,39.930,2.514,NaN,NaN,NaN,NaN,NaN,211.289,8.106
3,1,2010-02-26,46.630,2.561,NaN,NaN,NaN,NaN,NaN,211.320,8.106
4,1,2010-03-05,46.500,2.625,NaN,NaN,NaN,NaN,NaN,211.350,8.106


In [11]:
# ilk olarak train ve features dataframelerini ortak olan Store ve Date kolonlarına göre birleştirelim.
train_features = pd.merge(train, features, on=['Store', 'Date'], how='left')

In [12]:
# şimdi train_features ile stores dataframeini ortak olan Store değişkenine göre birleştirelim.
df = pd.merge(train_features, stores, on='Store', how='left')

In [13]:
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.500,False,42.310,2.572,NaN,NaN,NaN,NaN,NaN,211.096,8.106,A,151315
1,1,1,2010-02-12,46039.490,True,38.510,2.548,NaN,NaN,NaN,NaN,NaN,211.242,8.106,A,151315
2,1,1,2010-02-19,41595.550,False,39.930,2.514,NaN,NaN,NaN,NaN,NaN,211.289,8.106,A,151315
3,1,1,2010-02-26,19403.540,False,46.630,2.561,NaN,NaN,NaN,NaN,NaN,211.320,8.106,A,151315
4,1,1,2010-03-05,21827.900,False,46.500,2.625,NaN,NaN,NaN,NaN,NaN,211.350,8.106,A,151315


In [15]:
df.shape

(421570, 16)

In [16]:
# oluşturduğumuz yeni veri setini CSV dosyası olarak kaydedelim
df.to_csv('walmart.csv', index=False)

Elimizdeki dağınık veri setlerini uygun biçimde birleştirdik. Şimdi detaylarını inceleyelim.